# GeoSAM-Image-Encoder (Python package)

[![PyPI Version](https://img.shields.io/pypi/v/GeoSAM-Image-Encoder)](https://pypi.org/project/GeoSAM-Image-Encoder)
[![Downloads](https://static.pepy.tech/badge/GeoSAM-Image-Encoder)](https://pepy.tech/project/GeoSAM-Image-Encoder)


This package is part of the [Geo-SAM](https://github.com/coolzhao/Geo-SAM) project and is a standalone Python package that does not depend on QGIS. This package allows you to **encode remote sensing images into features that can be recognized by Geo-SAM using a remote server**, such as ``Colab``, ``AWS``, ``Azure`` or your own ``HPC``.

## Installation

Installing `GeoSAM-Image-Encoder` may directly install the CPU version of `PyTorch`. Therefore, it is recommended to install the appropriate version of `PyTorch` before installing `GeoSAM-Image-Encoder` in your machine. You can install the corresponding version based on the official PyTorch website:
<https://pytorch.org/get-started/locally/>

After installing PyTorch, you can install `GeoSAM-Image-Encoder` via pip.


In Colab, PyTorch is already built-in, so you can install it directly.

In [1]:
!pip uninstall -y GeoSAM-Image-Encoder geosam
!pip install -U git+https://github.com/atsyplenkov/GeoSAM-Image-Encoder.git@5466ccc

Found existing installation: GeoSAM-Image-Encoder 1.0.4
Uninstalling GeoSAM-Image-Encoder-1.0.4:
  Successfully uninstalled GeoSAM-Image-Encoder-1.0.4
  Cloning https://github.com/atsyplenkov/GeoSAM-Image-Encoder.git (to revision 5466ccc) to /tmp/pip-req-build-5jh2nru_
  Running command git clone --filter=blob:none --quiet https://github.com/atsyplenkov/GeoSAM-Image-Encoder.git /tmp/pip-req-build-5jh2nru_
  Running command git checkout -q 5466ccc
  Resolved https://github.com/atsyplenkov/GeoSAM-Image-Encoder.git to commit 5466ccc
  Preparing metadata (setup.py) ... done
  Created wheel for GeoSAM-Image-Encoder: filename=GeoSAM_Image_Encoder-1.0.4-py3-none-any.whl size=31413 sha256=d7e49042546829f798ef346d69770fb47d6e3ac41aa3e133f8fda57ab383aaed
  Stored in directory: /tmp/pip-ephem-wheel-cache-6h9714ri/wheels/67/3e/b6/4b94329e7f143fafe23dec385e2ef5b1b5d8e63ba8a7622100
Successfully built GeoSAM-Image-Encoder


Download example dataset and sam `vit_l` checkpoint

In [2]:
!wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth

--2026-02-09 14:21:15--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_l_0b3195.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.170.3.110, 3.170.3.104, 3.170.3.99, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.170.3.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1249524607 (1.2G) [binary/octet-stream]
Saving to: ‘sam_vit_l_0b3195.pth.2’

sam_vit_l_0b3195.pt  14%[=>                  ] 169.95M  49.3MB/s    eta 22s    ^C


Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Using Python

After install GeoSAM-Image-Encoder, you can import it using `geosam`

In [2]:
import geosam
from geosam import ImageEncoder

In [3]:
# check if gpu available
geosam.gpu_available()

True

Add CRS

In [2]:
import rasterio
from rasterio.crs import CRS

path = "/content/drive/MyDrive/Projects/Peskovsky/dom_tps_final.tif"

with rasterio.open(path, "r+") as ds:
    ds.crs = CRS.from_epsg(32638)

with rasterio.open(path) as ds:
    print("CRS after fix:", ds.crs)

CRS after fix: EPSG:32638


In [3]:
import inspect
import geosam.image_encoder as gie

src = inspect.getsource(gie.save_sam_feature)
print("has_export_fix:", "_resolution_suffix(" in src and "_res_{raster_ds.res:.6f}" not in src)

has_export_fix: True


#### Run by specify parameters directly

If you want to specify the parameters directly, you can run it like this:

In [4]:
import os, importlib
import geosam.image_encoder as gie

importlib.reload(gie)

checkpoint_path = "/content/sam_vit_l_0b3195.pth"
image_path = "/content/drive/MyDrive/Projects/Peskovsky/dom_tps_final.tif"
feature_dir = "/content/drive/MyDrive/Projects/Peskovsky/features"
os.makedirs(feature_dir, exist_ok=True)

enc = gie.ImageEncoder(checkpoint_path=checkpoint_path, model_type="vit_l", batch_size=1, gpu=True, gpu_id=0)
enc.encode_image(
    image_path=image_path,
    feature_dir=feature_dir,
    bands=[1, 2, 3],
    value_range=(0.0, 255.0),
    resolution=0.25,
    stride=512,
)

Initializing SAM model...


----------------------------------------------
     Start encoding image to SAM features
----------------------------------------------

Input Parameters:
----------------------------------------------
 Input data value range to be rescaled: (0.0, 255.0) (set by user)
 Image path: /content/drive/MyDrive/Projects/Peskovsky/dom_tps_final.tif
 Bands selected: ['1', '2', '3']
 Target resolution: 0.25
 Processing extent: [344044.07558097236, 4866170.96020715, 348030.82558097236, 4869733.46020715]
 Processing image size: (width 15947, height 14250)
----------------------------------------------


RasterDataset info 
----------------------------------------------
 filename_glob: dom_tps_final.tif, 
 all bands: ['1', '2', '3', '4'], 
 input bands: ['1', '2', '3'], 
 resolution: (0.25, 0.25), 
 bounds:                                                              minx  \
datetime                                                            
[1677-09-21 00:12:43.14522419

Encoding image: 100%|██████████| 837/837 [17:14<00:00,  1.24s/batch]

"Output feature path": /content/drive/MyDrive/Projects/Peskovsky/features
